In [1]:
import numpy as np
from os.path import join
import pandas as pd
from skimage.measure import regionprops_table
import matplotlib.pyplot as plt
from utils.nucStitch import fullNucImage16

In [ ]:
segmentationPath = "segmentations/inducedRainbow/old/red/5-60/aorta08/cellposeSegm_RedP5-60_aorta08_50Thres_final_seg.npy"
nucleiPath = "segmentations/inducedRainbow/final/nuclei/5-60/aorta08"
dfResult = pd.DataFrame()

In [ ]:
df = pd.DataFrame()
nuclei = np.load(join(nucleiPath, "Cropped_IMG-1_seg.npy"), allow_pickle=True).item()['masks']
yLength = nuclei.shape[0]
xLength = nuclei.shape[1]

# create segmentation of fully stitched aorta based on cropped images (glue crops back togther)
for i in range(16):
    number = i+1
    nuclei = np.load(join(nucleiPath, "Cropped_IMG-" + str(number) + '_seg.npy'), allow_pickle=True).item()['masks']
    row = int(i/4) + 1
    column  = (i%4) + 1
    prop_dict = regionprops_table(nuclei, properties = ('label', 'centroid'))
    df_help = pd.DataFrame(prop_dict)
    df_help['picture_id'] = number
    df_help['yCoord'] = df_help['centroid-0']*row
    df_help['xCoord'] = df_help['centroid-1']*column
    df = pd.concat([df, df_help], axis=0)

nucImage = fullNucImage16(nucleiPath)

In [26]:
#load rainbow segmentation
masks = np.load(segmentationPath, allow_pickle=True).item()['masks']

In [27]:
accuracy_value = 0.75
clusters = list()
dic = {}
dicNot = list()
for mask in np.unique(masks):
    if mask != 0:
        cluster = list()
        coord = np.where(masks == mask)
        segmentationNuc = np.unique(nucImage[coord])
        for j in segmentationNuc:
            if j != 0:
                #occurence of nucleus pixel in
                nucOccur = np.bincount(nucImage[coord])[j]
                #How many pixels does this nucleus have:
                coordLen = len(np.where(nucImage == j)[0])
                #check if most of nuclei values are in segmentation
                if(nucOccur >= accuracy_value*coordLen):
                    cluster.append(j)
        if cluster:
            dic[mask] = cluster
            clusters.append(cluster)
        else:
            dicNot.append(mask)

In [28]:
clusterSizes = []
for key in dic:
    clusterSizes.append(len(dic[key]))

In [36]:
resultdic = {"age": segmentationPath.split("/")[-3], "aorta": segmentationPath.split("/")[-2]}
unique, counts = np.unique(clusterSizes, return_counts=True)
for i in range(1,18):
    if i in unique:
        resultdic[i] = counts[np.where(unique==i)][0]
    else:
        resultdic[i] = 0

In [37]:
dfHelp = pd.DataFrame(resultdic, index = [0])

In [ ]:
dfResult = pd.concat([dfResult, dfHelp])

In [40]:
dfResult.to_csv("results/inducedRainbow/clusterSizes.csv")